In [12]:
import pandas as pd
import numpy as np
import re
import datetime as dt
from datetime import datetime
import json
from sqlalchemy.types import BigInteger, Time, Date, Float, Integer, Boolean, Text, TIMESTAMP

In [13]:
import sys

In [14]:
sys.path.append('python_scripts/ETL_python_scripts/')

In [15]:
import get_weather_from_google_search
import get_data_from_api
import ETL_utilities
import db_connection

In [16]:
city_names = ['Moscow', 'London']
columns = ['weather_id',
           'date',
           'time',
           'city_name',
           'temp_c',
           'temp_f']
db_df = pd.DataFrame(columns=columns)

In [17]:
#take a city
for city in city_names:
    #put city in question
    question = f'weather {city}'
    #make soup with question
    soup = get_data_from_api.soup_from_google_search(question)
    #make json from soup
    output_json = \
    get_weather_from_google_search.google_soup_to_json_weather(soup)
    #create a dictionary to save the results
    data_dict = {
    #create unique id
    'weather_id': ETL_utilities.create_unique_id_from_date(),
    #create date feature
    'date': datetime.now().date(),
    'time': datetime.now().time(),
    #data from json
    #load string json as dict and write in data dict
    'city_name': city,
    'temp_c': json.loads(output_json)['temperature_c'],
    'temp_f': json.loads(output_json)['temperature_f']}
    #convert dictionary to df for concat
    data_dict_df = pd.DataFrame([data_dict])
    db_df = pd.concat([db_df,data_dict_df])

In [18]:
db_df

,weather_id,date,time,city_name,temp_c,temp_f
0,202301-3110-2746-6e775ffd-45ba-4c76-b615-2ec6e...,2023-01-31,10:27:46.968339,Moscow,0.0,32.0
0,202301-3110-2748-5633ba0a-9a93-4a2c-a662-cdf5c...,2023-01-31,10:27:48.235802,London,9.0,48.0


In [19]:
engine = db_connection.engine
weather_dtypes = {'time': Time, 
        'date': Date, 
        'temp_c': Float, 
        'temp_f': Float}

In [20]:
weather_sql_query = '''
CREATE TABLE IF NOT EXISTS weather_prod(
weather_id TEXT NOT NULL PRIMARY KEY,
date DATE NOT NULL,
time TIME NOT NULL,
city_name TEXT NOT NULL,
temp_c FLOAT NOT NULL,
temp_f FLOAT NOT NULL);

INSERT INTO weather_prod (weather_id,
        date,
        time,
        city_name,
        temp_c,
        temp_f)
(select weather_id,
        date,
        time,
        city_name,
        temp_c,
        temp_f
from weather_tmp);
    '''

In [21]:
db_connection.load_df_to_tpm_table(db_df, 'weather',  weather_dtypes)

weather loading into DB...
weather_tmp loaded


In [22]:
db_connection.load_tmp_table_to_prod_table(weather_sql_query)

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "NOT"
LINE 8: temp_f FLOAT) NOT NULL;
                      ^

[SQL: 
CREATE TABLE IF NOT EXISTS weather_prod(
weather_id TEXT NOT NULL PRIMARY KEY,
date DATE NOT NULL,
time TIME NOT NULL,
city_name TEXT NOT NULL,
temp_c FLOAT NOT NULL,
temp_f FLOAT) NOT NULL;

INSERT INTO weather_prod (weather_id,
        date,
        time,
        city_name,
        temp_c,
        temp_f)
(select weather_id,
        date,
        time,
        city_name,
        temp_c,
        temp_f
from weather_tmp);
    ]
(Background on this error at: https://sqlalche.me/e/14/f405)